# Statistics

# import functions

In [1]:
from sqlays import export_sql, import_sql
from matheq import Cel_to_Kel, Kel_to_Cel, to_pot_temp
from mapscre import station_map, bathy_data, contour_ver, TS_diagram
from iscays import sum_up, isc_summary, depth_bin_interval, particle_bin_interval, isc_xlsx, read_isc
from ctdays import find_start_row, raw_ctd_to_df
from graphcre import deployment_constancy, vertical_each_org_distribution, stacked_vertical_distribution, comp_vertical_distribution
from matheq import calDist
from matplotlib import pyplot as plt
from matplotlib import gridspec
import pandas as pd
import numpy as np
import scipy.stats as stats
import os, glob, datetime, gsw, cmocean
from scipy.stats import linregress
from timeinfo import day_night

# import data

In [7]:
file_path = os.path.expanduser('~/Git/OCEANpy/data')
loki_df = export_sql('loki', 'all', 'awi_server')
ctd_df = export_sql('ctd', 'ctd_meta', 'local')
isc_stb = export_sql('isc', 'isc_meta', 'local')
isc_file_list = glob.glob(file_path+os.sep+'isc'+os.sep+'*.xlsx')

# data pre-processing

In [23]:
wm_dict = {10: (70, 95), 12: (70, 155), 14: (50, 215), 16: (100, 270)}
loki_stations = ('PS107_010-5', 'PS107_012-4', 'PS107_014-5', 'PS107_016-4')
loki_df_sel = loki_df.loc[loki_df['station'].str.contains('|'.join(loki_stations))]
print(list(loki_df_sel))
print(list(loki_df_sel['phylum'].drop_duplicates())) # Chaetognatha, Rhizaria
print(list(loki_df_sel['class'].drop_duplicates())) # Ostracoda
print(list(loki_df_sel['genus'].drop_duplicates()))
#[None, 'Oncaea', 'Pseudocalanus', 'Calanus', 'Microcalanus', 'Oithona', 'Metridia', 'Paraeuchaeta',
#'Scolecithricella', 'Eukrohnia', 'Themisto', 'Aglantha', 'Gaetanus', 'Thysanoessa', 'Scaphocalanus',
#'Spinocalanus', 'Microsetella', 'Heterorhabdus', 'Limacina', 'Temorites', 'Pleuromamma']
sp_col = ['']

['vessel', 'cruise', 'station', 'region', 'latitude', 'longitude', 'bottom_depth', 'date_time', 'loki_depth', 'temperatue', 'salinity', 'oxygen', 'fluorescence', 'manual_classification', 'developmental_stage', 'area_pixel', 'area_sqrmm', 'spec_length', 'spec_width', 'image_filename', 'animal', 'copepod', 'phylum', 'class', 'spec_order', 'family', 'genus', 'species']
[False, True]
[False, True]
['bubble', 'detritus', 'Crustacea', 'Chaetognatha', 'Animalia', 'Rhizaria', 'Cnidaria', 'Ctenophora', 'Mollusca', 'Annelida']
[None, 'Copepoda', 'Ostracoda', 'egg', 'Unknown marine', 'Radiozoa', 'Sagittoidea', 'Malacostraca', 'feces', 'Hydrozoa', 'Exuvia', 'Gastropoda', 'Polychaeta']
[None, 'Cyclopoida', 'Calanoida', 'Acantharia', 'Phragmophora', 'Amphipoda', 'Trachymedusae', 'Euphausiacea', 'Siphonophorae', 'Isopoda', 'Acantharea like', 'Harpacticoida', 'Euthecosomata']
[None, 'Oncaeidae', 'Clausocalanidae', 'Calanidae', 'Oithonidae', 'Metridinidae', 'Acantharia III', 'Euchaetidae', 'Scolecitric

# Linear Discriminant analysis